The goal of this notebook is to find a $\hat{c} \in \mathbb{R}^d$ (where $d$ is the number of constraints) such that
\begin{equation}
    \lVert h - A^T \hat{c} - K^T w \rVert_2^2 = \min_c \lVert h - A^T c - K^T w \rVert_2^2.
\end{equation}
We want to do this for the starting $w$ so that we are starting the ADMM endpoint optimization as close to a feasible point as possible.

In [31]:
from generate_opt_objects import A_b_generation, starting_point_generation
import numpy as np
import pickle
from scipy.optimize import minimize

In [8]:
# paths
OBS_FP = '../fixed_optimization_inputs/y_affine_corrected.npy'

# Current starting positions for $c$ and $w$

In [32]:
# get necessary objects
with open(OBS_FP, 'rb') as f:
    y_obs = np.load(f)

CONSTR_DIR = '/Users/mikestanley/Research/Carbon_Flux/optimization/data/sign_corrected'
A, b = A_b_generation(
    box_constraint_fp=CONSTR_DIR + '/scipy_bnds.pkl'
)

FUNC_FP = '/Users/mikestanley/Research/Carbon_Flux/optimization'
FUNC_FP += '/src_admm/data/carbon_flux/objects/na_june_functional.npy'
with open(FUNC_FP, 'rb') as f:
    h = np.load(f)

In [47]:
# define dimensions
m = y_obs.shape[0]
d = b.shape[0]
p = A.shape[1]

print(m, d, p)

28267 11120 26496


In [13]:
w_sp, c_sp, lambda_sp = starting_point_generation(
    m=m, d=d, p=p,
    random_seed=12345
)

In [15]:
np.where(c_sp != 0)

(array([], dtype=int64),)

# Getting $K^T w$ for the randomly selected $w$ start

In [17]:
HASH_DIR = '/Users/mikestanley/Research/Carbon_Flux/optimization/src_admm/data/carbon_flux/results/00'
with open(HASH_DIR + '/h_table_lep.pkl', 'rb') as f:
    KTw_dict = pickle.load(f)

In [25]:
# delete the 0 key
del KTw_dict[0]

In [29]:
# determine the key with the correct K^Tw evaluation
w_distances = dict()
min_dist = 1000.
min_dist_key = 0
for key in KTw_dict.keys():
    dist_key = np.linalg.norm(w_sp - KTw_dict[key]['w'])
    w_distances[key] = dist_key
    if dist_key < min_dist:
        min_dist_key = key
        min_dist = dist_key
        
print(f'Min dist: {min_dist} | Min dist key: {min_dist_key}')

Min dist: 5.513985464335888 | Min dist key: -7416693065889372294


Since we do not have the $K^Tw$ corresponding to the first randomly generated $w$, we see if this is possible with _some_ $w$.

# Trying for feasibility with _some_ $w$

Use the above $w$

In [33]:
KEY = -7416693065889372294

In [42]:
def obj_func(c, h=h, A=A, KTw=KTw_dict[KEY]['KTw']):
    return np.dot(h + A.T @ c - KTw, h + A.T @ c - KTw)

def obj_jac(c, h=h, A=A, KTw=KTw_dict[KEY]['KTw']):
    return 2 * A @ (h + A.T @ c - KTw)

In [46]:
# perform the optimization
c_opt_res = minimize(
    fun=obj_func, x0=c_sp, method='L-BFGS-B',
    jac=obj_jac,
    options={'iprint': 1}
)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        11120     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.77036D+02    |proj g|=  1.18356D+01

At iterate    1    f=  6.05888D+02    |proj g|=  9.83735D+00

At iterate    2    f=  3.40102D-27    |proj g|=  2.13163D-14

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
11120      2      4      1     0     0   2.132D-14   3.401D-27
  F =   3.4010185561787753E-027

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [45]:
c_opt_res['x'].max()

2.4150288105011026

# Feasibility with the actual initial $w$

In [55]:
# read in the randomly generated w
with open('../fixed_optimization_inputs/KTw_for_w_start.npy', 'rb') as f:
    KTw_0 = np.load(f)

In [56]:
def obj_func_0(c, h=h, A=A, KTw=KTw_0):
    return np.dot(h + A.T @ c - KTw, h + A.T @ c - KTw)

def obj_jac_0(c, h=h, A=A, KTw=KTw_0):
    return 2 * A @ (h + A.T @ c - KTw)

In [57]:
# perform the optimization
c_opt_res_0 = minimize(
    fun=obj_func_0, x0=c_sp, method='L-BFGS-B',
    jac=obj_jac_0,
    options={'iprint': 1}
)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        11120     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.19458D+03    |proj g|=  1.51110D+01

At iterate    1    f=  8.73952D+02    |proj g|=  1.29250D+01

At iterate    2    f=  2.07112D-27    |proj g|=  2.30926D-14

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
11120      2      4      1     0     0   2.309D-14   2.071D-27
  F =   2.0711166807940697E-027

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [58]:
print(f'Max c val: {c_opt_res_0["x"].max()}')
print(f'Objective: {c_opt_res_0["fun"]}')

Max c val: 7.555489540100109
Objective: 2.0711166807940697e-27


# Save the new starting points

In [54]:
SAVE_DIR = '/Users/mikestanley/Research/Carbon_Flux/optimization'
SAVE_DIR += '/src_admm/data/carbon_flux/results/01/intermediate_starts'

In [59]:
# w
with open(SAVE_DIR + "/cheyenne_stop_w_vec.npy", 'wb') as f:
    np.save(file=f, arr=w_sp)

# c
with open(SAVE_DIR + "/cheyenne_stop_c_vec.npy", 'wb') as f:
    np.save(file=f, arr=c_opt_res_0["x"])

# lambda
with open(SAVE_DIR + "/cheyenne_stop_lambda_vec.npy", 'wb') as f:
    np.save(file=f, arr=lambda_sp)